In [77]:
from openai import Client
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = Client(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_HOST"),
)

In [78]:
def gpt_completion(prompt, model="gpt-4"):
    message = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=0,
    )
    return response.choices[0].message.content
    # return response.choices[0].message.content

In [79]:
from crowdin_api import CrowdinClient


class FirstCrowdinClient(CrowdinClient):
    TOKEN = os.getenv("CROWDIN_API_KEY")
    TIMEOUT = 30  # Optional, sets http request timeout.
    RETRY_DELAY = 0.5  # Optional, sets the delay between failed requests 
    MAX_RETRIES = 5  # Optional, sets the number of retries
    PAGE_SIZE = 100  # Optional, sets default page size


crowdin = FirstCrowdinClient()

projects = crowdin.projects.list_projects()['data']
for project in projects:
    print(project['data']['name'], ":", project['data']['id'])


Casbin : 312853
Docusaurus v2 : 428890
Casnode : 479711
Casnode-Website : 481953
Casdoor-Site : 491513
Casdoor-Website : 493595
Casbin-Website : 521262


In [80]:
PROJECT_ID = 493595
PROJECT_NAME = "Casdoor-Website"

In [83]:
from pathlib import Path


class CrowdinFile:
    id: int
    project_id: int
    name: str
    path: str

    def __init__(self, project_id, _id, path):
        self.project_id = project_id
        self.id = _id
        self.path = path
        self.name = Path(path).name

    def __str__(self):
        return self.path


class CrowdinString:
    id: int
    file_id: int
    project_id: int
    text: str

    def __init__(self, project_id, file_id, _id, text):
        self.project_id = project_id
        self.file_id = file_id
        self.id = _id
        self.text = text

    def __str__(self):
        return self.text


class Translater:
    project_id: int
    project_name: str

    files = []
    prompt_zh = """
请将用三个反引号括起来的文本翻译为{lang}, 翻译时需要遵循下面的规则：
1. 文本中可能存在代码，你需要保留代码的原样。
2. 文本中可能包含 HTML 标签或者 HTML 转义字符，请不要翻译这些内容。
3. 如果你不确定如何翻译，请直接返回原文。
4. 请直接返回翻译后的文本或者原文，不要返回其他内容。

请翻译：
```{text}```
"""
    prompt = """
Please translate the text delimited by triple backticks into {lang}, adhering to the following rules:
1. The text may contain code, which you need to keep as is.
2. The text may include HTML tags or HTML escape characters, please do not translate these.
3. If you are unsure about how to translate, please return the original text.
4. Please return only the translated text or the original text, do not return anything else.

```{text}```
"""

    def __init__(self, project_id, project_name):
        self.project_id = project_id
        self.project_name = project_name

    def fetch_files(self):
        data = crowdin.source_files.list_files(self.project_id, limit=500)['data']
        for f in data:
            cf = CrowdinFile(self.project_id, f['data']['id'], f['data']['path'])
            allow_types = (".md", ".json", "mdx")
            if cf.name.endswith(allow_types):
                self.files.append(cf)
        return self.files

    def print_files(self):
        for file in self.files:
            print(file)

    def fetch_strings(self, file_id):
        data = crowdin.source_strings.list_strings(self.project_id, file_id, limit=500)['data']
        strings = []
        for s in data:
            cs = CrowdinString(self.project_id, file_id, s['data']['id'], s['data']['text'])
            strings.append(cs)
        return strings

    def translate(self, text, lang):
        res = gpt_completion(self.prompt.format(text=text, lang=lang))
        if res.startswith("```") and res.endswith("```"):
            return res[3:-3]
        else:
            return res

    def check_translation_if_existed(self, string_id, lang_id):
        res = crowdin.string_translations.list_string_translations(projectId=self.project_id, stringId=string_id,
                                                                   languageId=lang_id)
        if res['data']:
            return True
        else:
            return False

    def submit_translation(self, string_id, lang_id, translation):
        try:
            crowdin.string_translations.add_translation(projectId=self.project_id, stringId=string_id,
                                                        languageId=lang_id, text=translation)
        except Exception as e:
            print(e)

    def run(self, lang, lang_id):
        self.fetch_files()
        now = "/docs/permission/casbin-enforcement.md"
        files = []
        for file in self.files:
            if file.path == now:
                files = self.files[self.files.index(file):]
                break
        for file in files:
            print(f"正在翻译(id: {file.id})：", file.path)
            print("=========")
            strings = self.fetch_strings(file.id)
            for string in strings:
                existed = self.check_translation_if_existed(string.id,lang_id)
                if existed:
                    continue
                print(f"原文({string.id}):")
                print(string.text)
                translation = self.translate(string.text, lang)
                print("翻译:")
                print(translation)
                print("=========")
                self.submit_translation(string.id, lang_id, translation)


In [84]:
translater = Translater(PROJECT_ID, PROJECT_NAME)
translater.run("Chinese", "zh-CN")


正在翻译(id: 967)： /i18n/en/docusaurus-theme-classic/navbar.json
原文(47524):
Ecosystem
翻译:
生态系统
原文(47602):
For Enterprise
翻译:
适用于企业
原文(47604):
Hosting Plan (SaaS)
翻译:
托管计划（SaaS）
原文(47606):
Sign Up
翻译:
注册
原文(47608):
Login
翻译:
登录
正在翻译(id: 969)： /i18n/en/docusaurus-plugin-content-blog/options.json
正在翻译(id: 971)： /i18n/en/docusaurus-theme-classic/footer.json
原文(47496):
Copyright © 2024 Casdoor contributors.
翻译:
版权所有 © 2024 Casdoor 贡献者。
原文(47498):
Casdoor Logo
翻译:
Casdoor徽标
正在翻译(id: 973)： /i18n/en/code.json
原文(26405):
PDF download
翻译:
PDF下载
原文(26407):
We provide you with pdf for offline reading
翻译:
我们为您提供离线阅读的pdf
原文(26409):
English
翻译:
英语
原文(26411):
Chinese
翻译:
中文
原文(26649):
Created by
翻译:
由...创建
原文(26651):
Modified by
翻译:
修改者
原文(27835):
Discord
翻译:
Discord
原文(28253):
Direct link to {heading}
翻译:
直接链接到 {heading}
原文(28255):
Main
翻译:
主要
原文(28257):
Docs sidebar
翻译:
文档侧边栏
原文(31203):
Docs pages
翻译:
文档页面
原文(44767):
Translate this page
翻译:
翻译此页面
原文(47502):
Who's using Casdoor?
翻译:
谁在使用Casdoor？
原文(47504

KeyboardInterrupt: 